In [1]:
# This is an exmaple how to use spark sql for computing cosine similarity distance
df=spark.read.format("csv").option("header","true").option("inferSchema","true").load("/Users/kaicat/2.csv")

In [2]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- f1: integer (nullable = true)
 |-- f2: integer (nullable = true)
 |-- f3: integer (nullable = true)
 |-- f4: integer (nullable = true)
 |-- f5: integer (nullable = true)



In [3]:
from pyspark.ml.feature import VectorAssembler

In [4]:

featureCols =list(set(df.columns)-{'id'})

assembler = VectorAssembler().setInputCols(featureCols).setOutputCol("features")
df2 = assembler.transform(df)

In [5]:
df2.show()

df3=df2.select("features")
df3.show()



+---+---+---+---+---+---+--------------------+
| id| f1| f2| f3| f4| f5|            features|
+---+---+---+---+---+---+--------------------+
|  a|  1|  2|  3|  4|  5|[2.0,3.0,1.0,4.0,...|
|  b|  1|  2|  3|  4|  5|[2.0,3.0,1.0,4.0,...|
+---+---+---+---+---+---+--------------------+

+--------------------+
|            features|
+--------------------+
|[2.0,3.0,1.0,4.0,...|
|[2.0,3.0,1.0,4.0,...|
+--------------------+



In [6]:
from pyspark.mllib.util import *
df4=MLUtils.convertVectorColumnsFromML(df3,"features")

In [7]:
from pyspark.ml.linalg import *

rdd1 =df4.select("features").rdd.map(lambda row: row[0] )

In [8]:
from pyspark.mllib.linalg.distributed import RowMatrix
mat= RowMatrix(rdd1)
print(mat)
exact=mat.columnSimilarities()
exact.entries.collect()

[MatrixEntry(1, 4, 1.0),
 MatrixEntry(2, 3, 0.9999999999999998),
 MatrixEntry(2, 4, 0.9999999999999998),
 MatrixEntry(0, 1, 1.0),
 MatrixEntry(1, 2, 1.0),
 MatrixEntry(3, 4, 0.9999999999999998),
 MatrixEntry(0, 3, 0.9999999999999998),
 MatrixEntry(0, 4, 0.9999999999999998),
 MatrixEntry(1, 3, 1.0),
 MatrixEntry(0, 2, 0.9999999999999998)]